# Estimation of the observation error standard deviation

In [1]:
import pandas as pd
import numpy as np
from fatescal.tools import unit_conversion as uc
from fatescal.config import OBS_DATA_FILE_NAME, OBS_DATE_COLUMN_NAME, \
    PATH_TO_OBS_DATASET, FORCING_YR_START, FORCING_YR_END, VARIABLES_TO_ASSIMILATE

In [2]:
observations_df = pd.read_csv(PATH_TO_OBS_DATASET / OBS_DATA_FILE_NAME)
observations_df[OBS_DATE_COLUMN_NAME] = pd.to_datetime(
    observations_df[OBS_DATE_COLUMN_NAME],
    utc=True
).dt.tz_localize(None)  # Remove timezone information

start_date = np.datetime64(f"{FORCING_YR_START}-01-01")
end_date = np.datetime64(f"{FORCING_YR_END}-12-31")

date_mask = (observations_df[OBS_DATE_COLUMN_NAME] >= start_date) & \
    (observations_df[OBS_DATE_COLUMN_NAME] <= end_date)

observations_df = observations_df.loc[date_mask]
print(observations_df.head())
print(observations_df.tail())

                  samptime    GPP  ET_gapf
122692 2004-01-01 00:00:00  0.064    0.011
122693 2004-01-01 00:30:00 -0.157   -0.122
122694 2004-01-01 01:00:00 -0.357   -0.209
122695 2004-01-01 01:30:00 -0.154   -0.166
122696 2004-01-01 02:00:00  0.011   -0.177
                  samptime    GPP  ET_gapf
297984 2013-12-30 22:00:00 -1.086    0.127
297985 2013-12-30 22:30:00 -0.562    0.071
297986 2013-12-30 23:00:00 -0.846    0.120
297987 2013-12-30 23:30:00  0.227    0.259
297988 2013-12-31 00:00:00 -0.641    0.099


#### Convert to model unit

In [3]:
converted_observations_df = pd.DataFrame(columns=observations_df.columns)
converted_observations_df[OBS_DATE_COLUMN_NAME] = \
    observations_df[OBS_DATE_COLUMN_NAME]

for var_dict in VARIABLES_TO_ASSIMILATE:
    converted_observations_df[var_dict['Observed']['csv_col_name']] = \
        uc.convert_unit(
            values=observations_df[var_dict['Observed']['csv_col_name']],
            unit_in=var_dict['Observed']['unit'],
            unit_out=var_dict['CLM-FATES']['unit']
        )

converted_observations_df

,samptime,GPP,ET_gapf
122692,2004-01-01 00:00:00,7.687040e-10,1.981681e-07
122693,2004-01-01 00:30:00,-1.885727e-09,-2.197864e-06
122694,2004-01-01 01:00:00,-4.287927e-09,-3.765194e-06
122695,2004-01-01 01:30:00,-1.849694e-09,-2.990536e-06
122696,2004-01-01 02:00:00,1.321210e-10,-3.188705e-06
...,...,...,...
297984,2013-12-30 22:00:00,-1.304395e-08,2.287941e-06
297985,2013-12-30 22:30:00,-6.750182e-09,1.279085e-06
297986,2013-12-30 23:00:00,-1.016131e-08,2.161834e-06
297987,2013-12-30 23:30:00,2.726497e-09,4.665958e-06


### Define yearly quantile threshold

In [4]:
quantile = 0.99

def get_array_quantile(array):
    '''Custom function to calculate quantile'''
    return np.quantile(array, quantile)

yearly_quantiles_df = converted_observations_df.resample(
    '1Y', on=OBS_DATE_COLUMN_NAME
).apply(get_array_quantile).reset_index(drop=True)

mean_yearly_quantiles_df = yearly_quantiles_df.apply(lambda x: np.mean(x)) 
mean_yearly_quantiles_df

GPP        2.517929e-07
ET_gapf    8.758418e-05
dtype: float64

### Define error standard deviation and calculate absolute value

In [5]:
# No reliable values found in literature - based on expert opinion
error_std_as_fraction = 0.15
print(mean_yearly_quantiles_df * error_std_as_fraction)

GPP        3.776893e-08
ET_gapf    1.313763e-05
dtype: float64
